In [131]:
# ! pip install langchain

In [10]:
# Import packages 
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


import requests, json
import logging
import os
import io
import tempfile
from dotenv import load_dotenv

load_dotenv()

# You need to store your OPENAI API KEY and People Lab API Key in your environment
openai_api_key = os.environ["OPENAI_API_KEY"]
people_lab_api_key = os.environ["PEOPLE_LAB_API_KEY"]


############ Tools for Loading Longer data ############

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [11]:
############## Get Profile Data ################
url_input = "https://www.linkedin.com/in/jeremyscottross/"

if not (openai_api_key and people_lab_api_key):
    logging.ERROR("won't be able to generate response")


# Set your API key
API_KEY = people_lab_api_key

# Set the Person Enrichment API URL
PDL_URL = "https://api.peopledatalabs.com/v5/person/enrich"

# Create a parameters JSON object
PARAMS = {
    "api_key": people_lab_api_key,
    "profile": [url_input],
    "min_likelihood": 6
}


json_response = requests.get(PDL_URL, params=PARAMS).json()
content = json_response['data']

In [12]:
# Initialize Functions
def get_split_tests(profile):
    loader = TextLoader(profile) 
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=40)
    return text_splitter.split_documents(documents)



In [13]:
# Create an in-memory file-like object for profile
file_obj = io.StringIO(json.dumps(content))

# Write the contents of the file-like object to a temporary file
with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
    temp_file.write(file_obj.read())

# Pass the path to the temporary file to split text
split_texts = get_split_tests(temp_file.name)

# Remove the temporary file
os.remove(temp_file.name)

In [14]:
# Define your objective
objective_options = ['Sell a product', 'Recruit to job', 'Network']

# objective = objective_options[1]
objective = "tell me about this this person"

In [15]:
# Define the prompt template
template = """
    I have given you a prospects relevant linkedin profile data and my linked in data. Write me a message based on the objective

    Here are descriptions of some different input objectives:
    - Sell a product: Trying to schedule a meeting to sell this person a product
    - Recruit to job: Trying to recruit this person to work for your company
    - Network: Trying to expand connect with the person based on common interest
    
    Keep it under 300 characters.
    
    The INPUT OBJECTIVE is {objective}

    Below are the related documents about the Prospect:
    {prospect_related_documents}

"""

# Using vector store
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# define embeddings engine
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


# Don't need to load all the documents to the data only what is related to our objective
def get_related_documents(texts, embeddings, related_string):
    db = FAISS.from_documents(texts,embeddings)
    retriever = db.as_retriever()
    return retriever.get_relevant_documents(related_string)

prospect_info = get_related_documents(split_texts, embeddings, objective)



In [16]:
prompt = PromptTemplate(
    input_variables=["objective", "prospect_related_documents"],
    template=template,
)

prompt_with_input = prompt.format(objective=objective, prospect_related_documents=prospect_info)
llm = OpenAI(temperature=.7, openai_api_key=openai_api_key)
custom_message = llm(prompt_with_input)

custom_message

"\n\nHello! I'm messaging you about Jeremy Ross. He has over 10 years of experience in computer & network security, with roles in sales, business development, and management. He has worked at companies like Procter & Gamble, Gifts that Impress, and Flockjay. He is currently a Head of Demand and Sales and Founder at Revenue Genius. His profile also shows he is a member at Modern Sales Pros. Reach out and let's connect to discuss further!"